DEAP es compatible con Python 2.7 y 3.4 o superior
Recomiendo s que utilice easy_install o pip para instalar DEAP en su sistema. Los administradores de paquetes de Linux como apt-get, yum, etc. 
generalmente brindan una versión desactualizada.

In [2]:
!pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 4.2 MB/s eta 0:00:00


importación de las librerias, descripción detallada de cada una de ellas se puede encontrar en:
https://deap.readthedocs.io/en/master/api/index.html

Para el que no tiene conocimientos en la libreria numpy de python recomiendo realizar el siguiente tutorial.

https://web.stanford.edu/~schmit/cme193/lec/lec5.pdf

In [3]:
import operator
import math
import random
import numpy as np 
import pandas as pd 

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

In [4]:
samples=np.array([[0,0,0,0,0,0,0],
[0,0,0,0,0,1,0],
[0,0,0,0,1,0,0],
[0,0,0,0,1,1,0],
[0,0,0,1,0,0,0],
[0,0,0,1,0,1,0],
[0,0,0,1,1,0,0],
[0,0,0,1,1,1,0],
[0,0,1,0,0,0,1],
[0,0,1,0,0,1,1],
[0,0,1,0,1,0,1],
[0,0,1,0,1,1,1],
[0,0,1,1,0,0,1],
[0,0,1,1,0,1,1],
[0,0,1,1,1,0,1],
[0,0,1,1,1,1,1],
[0,1,0,0,0,0,0],
[0,1,0,0,0,1,0],
[0,1,0,0,1,0,0],
[0,1,0,0,1,1,0],
[0,1,0,1,0,0,1],
[0,1,0,1,0,1,1],
[0,1,0,1,1,0,1],
[0,1,0,1,1,1,1],
[0,1,1,0,0,0,0],
[0,1,1,0,0,1,0],
[0,1,1,0,1,0,0],
[0,1,1,0,1,1,0],
[0,1,1,1,0,0,1],
[0,1,1,1,0,1,1],
[0,1,1,1,1,0,1],
[0,1,1,1,1,1,1],
[1,0,0,0,0,0,0],
[1,0,0,0,0,1,0],
[1,0,0,0,1,0,1],
[1,0,0,0,1,1,1],
[1,0,0,1,0,0,0],
[1,0,0,1,0,1,0],
[1,0,0,1,1,0,1],
[1,0,0,1,1,1,1],
[1,0,1,0,0,0,0],
[1,0,1,0,0,1,0],
[1,0,1,0,1,0,1],
[1,0,1,0,1,1,1],
[1,0,1,1,0,0,0],
[1,0,1,1,0,1,0],
[1,0,1,1,1,0,1],
[1,0,1,1,1,1,1],
[1,1,0,0,0,0,0],
[1,1,0,0,0,1,1],
[1,1,0,0,1,0,0],
[1,1,0,0,1,1,1],
[1,1,0,1,0,0,0],
[1,1,0,1,0,1,1],
[1,1,0,1,1,0,0],
[1,1,0,1,1,1,1],
[1,1,1,0,0,0,0],
[1,1,1,0,0,1,1],
[1,1,1,0,1,0,0],
[1,1,1,0,1,1,1],
[1,1,1,1,0,0,0],
[1,1,1,1,0,1,1],
[1,1,1,1,1,0,0],
[1,1,1,1,1,1,1]])

In [5]:
values = np.array(samples[6])
pset = gp.PrimitiveSet("MAIN", 6)
pset.addPrimitive(np.logical_and, 2, name="vand")
pset.addPrimitive(np.logical_or, 2, name="vor")
pset.addPrimitive(np.logical_not, 1, name="vnot")
#pset.addPrimitive(cuadrado, 1, name="vpow2")
#pset.addPrimitive(np.cos, 1, name="vcos")
#pset.addPrimitive(np.sin, 1, name="vsin")
pset.addTerminal(1, name="vone")
pset.addTerminal(0, name='vzero')
pset.renameArguments(ARG0="p")
pset.renameArguments(ARG1="q")
pset.renameArguments(ARG2="r")
pset.renameArguments(ARG3="s")
pset.renameArguments(ARG4="t")
pset.renameArguments(ARG5="u")

In [6]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

In [7]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=2, max_=6)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

In [8]:
def evalSymbReg(individual):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the sum of squared difference between the expression
    # and the real function values : x**4 + x**3 + x**2 + x 
    diff = np.sum((func(samples[0], samples[1],samples[2],samples[3],samples[4],samples[5]) - values)**2) / len(samples)
    return np.sqrt(abs(diff)),

In [9]:
toolbox.register("evaluate", evalSymbReg)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=6)
toolbox.register('mutate', gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

In [10]:
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=7))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=7))


In [11]:
def main():
    random.seed(123456)

    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std) 
    stats.register("min", np.min)
    stats.register("max", np.max)

    algorithms.eaSimple(pop, toolbox, 0.6, 0.8, 100, stats, halloffame=hof)
   
    return pop, stats, hof

if __name__ == "__main__":
    p,s,h=main()
    sol=(h[0])
    error=evalSymbReg(sol)
    print('modelo:',sol)
    print('error:',error)
    func = toolbox.compile(expr=sol)
    #print(func(0,0,0,1,1,0))
    test=[func(*row[:6]) for row in samples]
    print(test)

gen	nevals	avg     	std      	min	max     
0  	100   	0.224693	0.0671804	0  	0.306186
1  	87    	0.195999	0.0546878	0.125	0.306186
2  	94    	0.18842 	0.0527191	0.125	0.306186
3  	90    	0.181226	0.0512424	0.125	0.306186
4  	91    	0.174261	0.0478457	0.125	0.306186
5  	96    	0.156691	0.0412065	0.125	0.279508
6  	94    	0.152402	0.0473545	0    	0.279508
7  	97    	0.157741	0.0541322	0    	0.306186
8  	90    	0.149719	0.0508354	0    	0.306186
9  	95    	0.159167	0.0635669	0    	0.306186
10 	92    	0.151404	0.0563081	0    	0.306186
11 	93    	0.138093	0.0558369	0    	0.306186
12 	90    	0.129242	0.0626221	0    	0.279508
13 	97    	0.122563	0.0864777	0    	0.279508
14 	90    	0.0732058	0.0890066	0    	0.306186
15 	93    	0.0571036	0.0848995	0    	0.279508
16 	97    	0.0704563	0.0978118	0    	0.279508
17 	86    	0.0617586	0.093198 	0    	0.306186
18 	93    	0.0735447	0.0921815	0    	0.279508
19 	92    	0.0613215	0.0926468	0    	0.306186
20 	97    	0.054178 	0.082172 	0    	0.306186
21 	94 